In [2]:
import pandas as pd

from google.colab import files
uploaded = files.upload()

import io
post_covid_df = pd.read_csv(io.BytesIO(uploaded['disinfecting_wipe_Post_Covid.csv']))

Saving disinfecting_wipe_Post_Covid.csv to disinfecting_wipe_Post_Covid.csv


In [3]:
post_covid_df.head()

,product_category,product_url,review_date,Date,handle,rating,helpfulness_rating,review
0,disinfecting_wipes,https://www.amazon.com/dp/B00Q70RCW6,11/2/2020,Post_Covid,Swiszms,5,NaN,(4) 80 count canisters are a great value for t...
1,disinfecting_wipes,https://www.amazon.com/dp/B00Q70RCW6,1/6/2021,Post_Covid,alundra,1,NaN,4 out of 4 arrived smashed and damaged. There ...
2,disinfecting_wipes,https://www.amazon.com/dp/B00Q70RCW6,3/10/2020,Post_Covid,Mandy,5,NaN,A absolutely must have for all households and ...
3,disinfecting_wipes,https://www.amazon.com/dp/B00Q70RCW6,10/23/2020,Post_Covid,Hui,3,NaN,A little expensive.I bought from Walmart.It’s ...
4,disinfecting_wipes,https://www.amazon.com/dp/B00Q70RCW6,1/23/2021,Post_Covid,D. M. Wilkerson,5,NaN,A much needed household cleaning product for o...


In [4]:
post_covid_df = post_covid_df.drop(columns=['product_url', 'helpfulness_rating'], axis=1)

In [5]:
post_covid_df['rating'] = post_covid_df['rating'].apply(lambda z:'positive' if z>3 else 'negative')

In [6]:
post_covid_df.head()

,product_category,review_date,Date,handle,rating,review
0,disinfecting_wipes,11/2/2020,Post_Covid,Swiszms,positive,(4) 80 count canisters are a great value for t...
1,disinfecting_wipes,1/6/2021,Post_Covid,alundra,negative,4 out of 4 arrived smashed and damaged. There ...
2,disinfecting_wipes,3/10/2020,Post_Covid,Mandy,positive,A absolutely must have for all households and ...
3,disinfecting_wipes,10/23/2020,Post_Covid,Hui,negative,A little expensive.I bought from Walmart.It’s ...
4,disinfecting_wipes,1/23/2021,Post_Covid,D. M. Wilkerson,positive,A much needed household cleaning product for o...


In [7]:
import nltk

nltk.download('punkt')

import string
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [8]:
post_covid_df['review'] = post_covid_df['review'].astype(str)

In [9]:
from nltk.tokenize import word_tokenize

for item in post_covid_df['review']:
  words = word_tokenize(item.replace('\n',' '))

In [10]:
clean = [word for word in words if word not in set(string.punctuation)]

In [11]:
characters_to_remove = ["''",'``','!',"'"]
clean_words = [word for word in clean if word not in set(characters_to_remove)]

In [12]:
nltk.download('stopwords')

from nltk.corpus import stopwords
english_stops = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [13]:
clean = [word for word in clean_words if word not in english_stops]

In [14]:
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [15]:
reviews = [(list(clean), rating)
              for review_text in post_covid_df['review']
              for rating in post_covid_df['rating']]

random.shuffle(reviews)

In [16]:
clean = [wordnet_lemmatizer.lemmatize(word) for word in clean]

In [17]:
all_words = nltk.FreqDist(word.lower() for word in clean)
word_features = list(all_words)

def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

In [18]:
split = 0.7

featuresets = [(document_features(d), c) for (d,c) in reviews]

train_set, test_set = featuresets[int(round(len(featuresets)*split,0))-1:], featuresets[:int(round(len(featuresets)*split,0))-1]

nb_classifier = nltk.NaiveBayesClassifier.train(train_set)

In [19]:
print(nltk.classify.accuracy(nb_classifier, test_set))

0.8157820154662132


In [20]:
nb_classifier.show_most_informative_features(n=15)

Most Informative Features
           contains(lid) = True           negati : positi =      1.0 : 1.0
          contains(wipe) = False          negati : positi =      1.0 : 1.0
           contains(one) = False          negati : positi =      1.0 : 1.0
      contains(squished) = True           negati : positi =      1.0 : 1.0
          contains(come) = True           negati : positi =      1.0 : 1.0
          contains(pack) = True           negati : positi =      1.0 : 1.0
